In [1]:
import pandas as pd
import time
from requests_html import AsyncHTMLSession
import asyncio
# from requests_html import HTMLSession
import requests
from bs4 import BeautifulSoup as bs
import traceback
import numpy as np
from scipy.stats import linregress
#from concurrent.futures import ThreadPoolExecutor
import json

C:\Users\Tanuj\AppData\Local\Temp\ipykernel_26840\734134811.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
session=AsyncHTMLSession()

#Loading ticker data from the CSV
df=pd.read_csv("data.csv")
df

,Symbol,name
0,A,agilent-technologies
1,AA,alcoa
2,AAC,ata-creativity-global
3,AACG,ata-creativity-global
4,AADI,aadi-bioscience
...,...,...
4952,ZVRA,zevra-therapeutics
4953,ZWS,zurn-water-solutions
4954,ZYME,zymeworks
4955,ZYNE,zynerba-pharmaceuticals


In [8]:
total=len(df['Symbol'])
symbol=""
perpetual_growth_rate=0.025

In [9]:

#-----------------------------------------------------------------------------------#
#                           Required custom functions                               #
#-----------------------------------------------------------------------------------#
def to_float(valueo:str):
    try:
        value=valueo.strip().replace(" ","").replace(",","").replace("$","").replace("%","")
        if "M" in value:
            value=float(value.replace("M",""))*1000000
            return value
        elif "B" in value:
            value=float(value.replace("B",""))*1000000000
            return value
        elif "T" in value:
            value=float(value.replace("T",""))*1000000000000
            return value
        else:
            return float(value)
    except:
        return 0
#------------------------------------------------------------------------------#
def to_int(valueo:str):
    try:
        value=valueo.strip().replace(" ","").replace(",","").replace("$","").replace("%","")
        if "M" in value:
            value=int(value.replace("M",""))*1000000
            return value
        elif "B" in value:
            value=int(value.replace("B",""))*1000000000
            return value
        elif "T" in value:
            value=int(value.replace("T",""))*1000000000000
            return value
        else:
            return int(value)
    except:
        return 0
#--------------------------------------------------------------------------------------#
async def get_cashflow():
    data={}
    try:
        res=await session.get(f"https://finance.yahoo.com/quote/{symbol}/cash-flow")
        await res.html.arender(timeout=10000)
        soup=bs(res.content,'html.parser')
        row = soup.find('div', {'title': 'Free Cash Flow'}).findPrevious('div',{'data-test': 'fin-row'})
        columns=row.findAll('div', {'data-test': 'fin-col'})

        data={
            "FCF 2023":to_int(columns[1].text)*1000,
            "FCF 2022":to_int(columns[2].text)*1000,
            "FCF 2021":to_int(columns[3].text)*1000,
            "FCF 2020":to_int(columns[4].text)*1000,
        }

        return data
    except:
        return data
#--------------------------------------------------------------------------------------#
    
async def get_balanceSheet():
    data={}
    try:
        res=await session.get(f"https://finance.yahoo.com/quote/{symbol}/balance-sheet")
        await res.html.arender(timeout=10000)
        soup=bs(res.content,'html.parser')
        TotalDebt = soup.find('div', {'title': 'Total Debt'}).findPrevious('div',{'data-test': 'fin-row'})
        TotalDebt=TotalDebt.findAll('div', {'data-test': 'fin-col'})[1].text
        # row = soup.findAll('div', {'data-test': 'fin-row'})[10]
        # columns=row.findAll('div', {'data-test': 'fin-col'})

        data={
            "Total Debt":to_int(TotalDebt)*1000,
        }

        return data
    except:
        return data
#--------------------------------------------------------------------------------------#
async def get_analysis():
    data={}
    try:
        res=await session.get(f"https://finance.yahoo.com/quote/{symbol}/analysis")
        await res.html.arender(timeout=10000)
        soup=bs(res.content,'html.parser')
        analysis = soup.find('section', {'data-test': 'qsp-analyst'})
        growth=analysis.find_all("table")[-1]
        rows=growth.find("tbody").find_all("tr")
        data={}
        data["Next Year"]=rows[3].find_all("td")[1].text
        data["Next 5Year"]=rows[4].find_all("td")[1].text
        return data
    except:
        return data
#--------------------------------------------------------------------------------------#
async def get_stats():
    data={}
    try:
        res=await session.get(f"https://finance.yahoo.com/quote/{symbol}/key-statistics")
        await res.html.arender(timeout=10000)
        soup=bs(res.content,'html.parser')
        stats = soup.find('section', {'data-test': 'qsp-statistics'})
        share_stats=stats.find_all("table")[2]
        share_out=share_stats.find("tbody").find_all("tr")[2].find_all("td")[1]

        ROE_stats=stats.find_all("table")[6]
        ROE_out=ROE_stats.find("tbody").find_all("tr")[1].find_all("td")[1]
        data={}
        data["Share Outstandings"]=to_float(share_out.text)
        data["Return on Equity (ttm)"]=to_float(ROE_out.text)/100
        return data
    except:
        return data
#--------------------------------------------------------------------------------------#
async def get_financials():
    data={}
    try:
        res=await session.get(f"https://finance.yahoo.com/quote/{symbol}/financials")
        await res.html.arender(timeout=10000)
        soup=bs(res.content,'html.parser')
        financials = soup.find('section', {'data-test': 'qsp-financial'})
        row = financials.find('div', {'title': 'Diluted EPS'}).findPrevious('div',{'data-test': 'fin-row'})
        columns=row.findAll('div', {'data-test': 'fin-col'})
        InterestExpense = financials.find('div', {'title': 'Interest Expense'}).findPrevious('div',{'data-test': 'fin-row'})
        InterestExpense=InterestExpense.findAll('div', {'data-test': 'fin-col'})[1].text
        IncomeTaxExpense=financials.find('div', {'title': 'Tax Provision'}).findPrevious('div',{'data-test': 'fin-row'})
        IncomeTaxExpense=IncomeTaxExpense.findAll('div', {'data-test': 'fin-col'})[1].text
        Incomebeforetax=financials.find('div', {'title': 'Pretax Income'}).findPrevious('div',{'data-test': 'fin-row'})
        Incomebeforetax=Incomebeforetax.findAll('div', {'data-test': 'fin-col'})[1].text
        data={
            "Eps Diluted 2024":float(columns[0].text),
            "Eps Diluted 2023":float(columns[1].text),
            "Eps Diluted 2022":float(columns[2].text),
            "Eps Diluted 2021":float(columns[3].text),
            "Eps Diluted 2020":float(columns[4].text),
        }
        if data['Eps Diluted 2020']>0 and data['Eps Diluted 2021']>0 and data['Eps Diluted 2022']>0 and data['Eps Diluted 2023']>0 and data['Eps Diluted 2024']>0:
            pass
        else:
            data['reason']="negative EPS"
        data['Eps Diluted 2023%']=(data['Eps Diluted 2023']-data['Eps Diluted 2022'])/data['Eps Diluted 2022']
        data['Eps Diluted 2022%']=(data['Eps Diluted 2022']-data['Eps Diluted 2021'])/data['Eps Diluted 2021']
        data['Eps Diluted 2021%']=(data['Eps Diluted 2021']-data['Eps Diluted 2020'])/data['Eps Diluted 2020']
        sum=0
        # for each in data:
        #     sum=sum+data[each]
        # average_eps=sum/len(data)
        data['EPS rate%']=((data['Eps Diluted 2023%']+data['Eps Diluted 2022%']+data['Eps Diluted 2021%'])/3)

        data['Interest Expense']=to_int(InterestExpense)*1000
        data['Income Tax Expense']=to_int(IncomeTaxExpense)*1000
        data['Income before tax']=to_int(Incomebeforetax)*1000
        data['Effective tax rate (in %)']=int(IncomeTaxExpense.replace(",",""))/int(Incomebeforetax.replace(",",""))/100
        return data
    except:
        return data
#--------------------------------------------------------------------------------------#



async def get_data_summary():
    data={}
    try:
        res=await session.get(f"https://finance.yahoo.com/quote/{symbol}")
        await res.html.arender(timeout=10000)
        soup=bs(res.content,'html.parser')
        data={
            "Price":to_float(get_price(soup)),
            "PE Ratio":to_float(get_pe_ratio(soup)),
            "EPS(TTM)":to_float(get_eps_ratio(soup)),
            "Beta":to_float(get_beta(soup)),
            "Market Cap":to_float(get_market_cap(soup))
        }
        return data
    except:
        return data
#--------------------------------------------------------------------------------------#
def get_price(soup):
    try:
        td_tag = soup.find('td', {'data-test': 'PREV_CLOSE-value'})
        return td_tag.text
    except:
        return "N/A"
#--------------------------------------------------------------------------------------#
def get_pe_ratio(soup):
    try:
        td_tag = soup.find('td', {'data-test': 'PE_RATIO-value'})
        return td_tag.text
    except:
        return "N/A"
#--------------------------------------------------------------------------------------#
def get_eps_ratio(soup):
    try:
        td_tag = soup.find('td', {'data-test': 'EPS_RATIO-value'})
        return td_tag.text
    except:
        return "N/A"
#--------------------------------------------------------------------------------------#
def get_beta(soup):
    try:
        td_tag = soup.find('td', {'data-test': 'BETA_5Y-value'})
        return td_tag.text
    except:
        return "N/A"
#--------------------------------------------------------------------------------------#
def get_market_cap(soup):
    try:
        td_tag = soup.find('td', {'data-test': 'MARKET_CAP-value'})
        return td_tag.text
    except:
        return "N/A"
#--------------------------------------------------------------------------------------#

async def get_cash_on_hand():
    try:
        s=ticker[f"{symbol}"].lower()
        headers = {
    # 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'Accept-Encoding': 'gzip, deflate, br, zstd',
    'Accept-Language': 'en-IN,en-GB;q=0.9,en-US;q=0.8,en;q=0.7',
    # 'Cache-Control': 'max-age=0',
    # 'Cookie': 'usprivacy=1---; _jsuid=3299131652; _gd_visitor=fa33abaa-1039-4531-8883-ad6bbe144f25; _gd_svisitor=65f06e68b23a1c00d1a0ad64e80000009d480100; __qca=P0-717079984-1708790328855; cnx_userId=2e953735fd05415cbac2a73f839f4337; _cc_id=a74bb47423820f64ef9eb6e6192b979f; panoramaId_expiry=1709395152451; panoramaId=98ed3cc483515500cf7e078beefc4945a70221d4cf3341b92af7778bac4a36b3; panoramaIdType=panoIndiv; cf_clearance=d_4Z3s3mOj_nQquUAkC5Q.JRF8vJ.Z7Y6Z8PlSlynLY-1709042877-1.0-AWpwemo1yV5Rkq8HrkFiDH/mX+/M0RrnTHvWAGN/3udwiSL1RUt7Ww+uazNdW3sP8KsiYaBCZLSpJ+NDIRpx4oM=; _gid=GA1.2.1071979747.1709132954; _ga=GA1.1.1218498466.1708790319; cto_bundle=qFZ19F9TZSUyRjYyTHBNQ3BjRlI4V0hzVG1HUXplSEFVRXhhTWNhNDU0Tmo5MFRBa2xPZWxQYTl3VFM0R2wwSXpQZiUyQkx2OVVPblVLdmNyenQlMkZqRnhhSEFQNU5EeDFDUTlzbTBDNjhER252OFUzQllkeTZnR1olMkZNOWhzJTJCS0thcU5pZXBvRWRiOFJyZHBBNTVrQmxCa0dRZDFPaUhocm9jSE1hNmczRm9mUW1rZExEY2VFJTNE; _gd_session=c3a59520-bdeb-4aa3-8b66-3b6579e3289e; __cf_bm=q2A8MMRywMty0bumNcft945PJdBWD_xkQ_cIfMEmKdw-1709135150-1.0-AVOVwzHVlpEF8yrOy1kY7die3MoHWDWpEZ2ErcSckdbd6tgToPAmTYoeCjhsMDLJYwA4IhGLIIHacLOkRVnwvsY=; _ga_3KL0LYERBH=GS1.1.1709132122.6.1.1709135153.54.0.0; IC_ViewCounter_www.macrotrends.net=2; _awl=2.1709135154.5-94859f662a79cc15aeb732e4e6c6195c-6763652d617369612d6561737431-0; __gads=ID=c5dccd64ce225151:T=1708790343:RT=1709135780:S=ALNI_Mbm58BmNA1bllCyPNA0nWxfkUu1Dg; __gpi=UID=00000d13de8a691d:T=1708790343:RT=1709135780:S=ALNI_MZR6K4TSMO0Pn_3Pd1GjxZu5Z1K-A; __eoi=ID=8bdae96c8575d5cb:T=1708790343:RT=1709135780:S=AA-Afja5M66NSTrC2kzWZqTzXBCk; __cf_bm=LhvvyEHF.koOjkT5Pb18zYeaCNQShMRWb5FYgiaHGCk-1709143181-1.0-AWbTZU8EfeGzz9x1vY8b8q/3HSqyWcLSBKaPGkc4zEFoTQ8G5iCsYVouGAjG83TaOqGrFp+CN+tXsHozERPK7IU=',
    # 'If-Modified-Since': 'Wed, 28 Feb 2024 10:57:34 GMT',
    'Referer': f'https://www.macrotrends.net/stocks/charts/{symbol}/{s}/other-current-assets',
    'Sec-Ch-Ua': '"Not A(Brand";v="99", "Google Chrome";v="121", "Chromium";v="121"',
    'Sec-Ch-Ua-Mobile': '?0',
    'Sec-Ch-Ua-Platform': '"Windows"',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-User': '?1',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36'
    }
        url=f"https://www.macrotrends.net/stocks/charts/{s}/cash-on-hand"
        print(url)
        session1 = requests.Session()
        request = session1.get(url,headers=headers)
        cookies = dict(request.cookies)
        j=0
        while True:
            j+=1
            res= await session.get(url,headers=headers,timeout=10000,cookies=cookies)
            await res.html.arender(timeout=10000)
            cash_on_hand=bs(res.content,"html.parser")
            table=cash_on_hand.find("table",{'class':'historical_data_table table'})
            if table is not None:
                cash=table.find("tbody").find('tr').find_all('td')[1].text
                data={}
                data['Cash/ Cash Equivalent']=to_int(cash)*1000000
                print(data)
                return data
            if j>10:
                data={}
                data['Cash/ Cash Equivalent']=0
                return data
            await asyncio.sleep(1)
            
            
    except Exception as e:
        print("Cash/cash equivalent error",e,res.content)
        data={}
        data['Cash/ Cash Equivalent']=0
        return data
#--------------------------------------------------------------------------------------#

result_bl=[]
result=[]

In [11]:

#Iterate over n number of stocks from the stocks csv file
for i in range(total):
#for i in range(10):
    print("currently processing:",i)
    try:
        symbol=df['Symbol'][i]
        #Getting data from yahoo finance
        print(df['Symbol'][i],df["name"][i])
        time.sleep(1)
        # print(session.run(get_stats))
        # out=session.run(get_data_summary)+session.run(get_cashflow)+session.run(get_stats)+session.run(get_analysis)+session.run(get_financials)+session.run(get_balanceSheet)+session.run(get_cash_on_hand)
        out=session.run(get_data_summary,get_cashflow,get_stats,get_analysis,get_financials,get_balanceSheet,get_cash_on_hand)
        #out=asyncio.run(get_data_summary(),get_cashflow(),get_stats(),get_analysis(),get_financials(),get_balanceSheet(),get_cash_on_hand())
        
        data={
            "Stock Name":df["name"][i].upper(),
            "Symbol":df['Symbol'][i]
            }
        url = f"https://production.dataviz.cnn.io/quote/forecast/{symbol}"
        url2 = f"https://production.dataviz.cnn.io/quote/analystratings/{symbol}"
        headers = {'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.106 Safari/537.36',
                   'accept-language': 'en,gu;q=0.9,hi;q=0.8',
                   'accept-encoding': 'gzip, deflate, br'}
        
        #Building Data for EPS and DCF
        for each in out:
            data=data|each
        data['FCF 2024']=data['FCF 2023']*(1+(float(data['Next Year'].removesuffix("%"))/100))
        data['FFCF 2024']=data['FCF 2023']*(1+(float(data['Next 5Year'].removesuffix("%"))/100))
        data['FFCF 2025']=data['FFCF 2024']*(1+(float(data['Next 5Year'].removesuffix("%"))/100))
        data['FFCF 2026']=data['FFCF 2025']*(1+(float(data['Next 5Year'].removesuffix("%"))/100))
        data['FFCF 2027']=data['FFCF 2026']*(1+(float(data['Next 5Year'].removesuffix("%"))/100))
        data['FFCF 2028']=data['FFCF 2027']*(1+(float(data['Next 5Year'].removesuffix("%"))/100))
        data['Cost of Debt (in %)']=(data['Interest Expense']/data['Total Debt'])
        data['Cost of debt after tax(in %)']=(float(data['Cost of Debt (in %)'])*(1+float(data['Effective tax rate (in %)'])))
        data['Risk Free rate( in %)']=0.0425
        data['Market Return (in %)']=0.09
        data['Cost of equity (in %)']=(data['Risk Free rate( in %)']+((data['Beta'])*(data['Market Return (in %)']-data['Risk Free rate( in %)'])))
        data['Total']=data['Total Debt']+data['Market Cap']
        data['Weigh of debt(in %)']=(data['Total Debt']/data['Total'])
        data['Weight of equity(in %)']=(data['Market Cap']/data['Total'])
        data['WACC']=(float(data['Cost of Debt (in %)'])*(data['Weigh of debt(in %)']))+(data['Cost of equity (in %)']*data['Weight of equity(in %)'])
        data['Terminal Value']=data['FFCF 2028']*(1+perpetual_growth_rate)/(data['WACC']-perpetual_growth_rate)
        data['PV of FFCF 2024']=data['FFCF 2024']/(1+data['WACC'])**1
        data['PV of FFCF 2025']=data['FFCF 2025']/(1+data['WACC'])**2
        data['PV of FFCF 2026']=data['FFCF 2026']/(1+data['WACC'])**3
        data['PV of FFCF 2027']=data['FFCF 2027']/(1+data['WACC'])**4
        data['PV of FFCF 2028']=data['FFCF 2028']/(1+data['WACC'])**5
        data['PV of Terminal Value']=data['Terminal Value']/(1+data['WACC'])**6
        data['Sum of PV of FFCF']=data['PV of FFCF 2024']+data['PV of FFCF 2025']+data['PV of FFCF 2026']+data['PV of FFCF 2027']+data['PV of FFCF 2028']+data['PV of Terminal Value']
        data['Equity Value']=data['Sum of PV of FFCF']+data['Cash/ Cash Equivalent']-data["Total Debt"]
        data['Price per share(Intrinsic Value via DCF)']=data['Equity Value']/data["Share Outstandings"]
        data['Perpetual Growth rate']=perpetual_growth_rate
        data["Graham Formula / Intrinsic value using EPS"]=(8.5+(data['EPS rate%'])*100)*data['EPS(TTM)']
        data['Over/Undervalue %']=(data['Price']-data['Graham Formula / Intrinsic value using EPS'])/data['Graham Formula / Intrinsic value using EPS']
        try:
            session1 = requests.Session()
            request = session1.get(url,headers=headers)
            request2 = session1.get(url2,headers=headers)
            cookies = dict(request.cookies)
            cookies2 = dict(request2.cookies)
            response = session1.get(url, headers=headers, cookies=cookies).json()
            response2 = session1.get(url2, headers=headers, cookies=cookies2).json()
            rdata = json.dumps(response)
            rdata2 = json.dumps(response2)
            json_data = json.loads(rdata)
            json_data2 = json.loads(rdata2)
            data1 = json_data[0]
            data2 = json_data2[0]
            data['High']=data1['high_target_price']
            data['Median']=data1['median_target_price']
            data['Low']=data1['low_target_price']
            data['Last']=data1['current_stock_price']
            data['Sym']=data1['symbol']
            data['Anal']=data2["num_of_analysts"]
        except:
            pass
        print(data)
        result.append(data)
    
        #Building data for best line method
        data_bl=data.copy()
        data_bl['FCF 2024']=data_bl['FCF 2023']*(1+(float(data_bl['Next Year'].removesuffix("%"))/100))
        years=np.array([2020,2021,2022,2023])
        values=np.array([int(data_bl['FCF 2020']),int(data_bl['FCF 2021']),int(data_bl['FCF 2022']),int(data_bl['FCF 2023'])])
        slope,intercept,r_value,p_value,std_err=linregress(years,values)
        data_bl['M']=slope
        data_bl['FFCF 2024']=data_bl['FCF 2023']+slope
        data_bl['FFCF 2025']=data_bl['FFCF 2024']+slope
        data_bl['FFCF 2026']=data_bl['FFCF 2025']+slope
        data_bl['FFCF 2027']=data_bl['FFCF 2026']+slope
        data_bl['FFCF 2028']=data_bl['FFCF 2027']+slope
        data_bl['Cost of Debt (in %)']=(data_bl['Interest Expense']/data_bl['Total Debt'])
        data_bl['Cost of debt after tax(in %)']=(float(data_bl['Cost of Debt (in %)'])*(1+float(data_bl['Effective tax rate (in %)'])))
        data_bl['Risk Free rate( in %)']=0.0425
        data_bl['Market Return (in %)']=0.09
        data_bl['Cost of equity (in %)']=(data_bl['Risk Free rate( in %)']+((data_bl['Beta'])*(data_bl['Market Return (in %)']-data_bl['Risk Free rate( in %)'])))
        data_bl['Total']=data_bl['Total Debt']+data_bl['Market Cap']
        data_bl['Weigh of debt(in %)']=(data_bl['Total Debt']/data_bl['Total'])
        data_bl['Weight of equity(in %)']=(data_bl['Market Cap']/data_bl['Total'])
        data_bl['WACC']=(float(data_bl['Cost of Debt (in %)'])*(data_bl['Weigh of debt(in %)']))+(data_bl['Cost of equity (in %)']*data_bl['Weight of equity(in %)'])
        data_bl['Terminal Value']=data_bl['FFCF 2028']*(1+perpetual_growth_rate)/(data_bl['WACC']-perpetual_growth_rate)
        data_bl['PV of FFCF 2024']=data_bl['FFCF 2024']/(1+data_bl['WACC'])**1
        data_bl['PV of FFCF 2025']=data_bl['FFCF 2025']/(1+data_bl['WACC'])**2
        data_bl['PV of FFCF 2026']=data_bl['FFCF 2026']/(1+data_bl['WACC'])**3
        data_bl['PV of FFCF 2027']=data_bl['FFCF 2027']/(1+data_bl['WACC'])**4
        data_bl['PV of FFCF 2028']=data_bl['FFCF 2028']/(1+data_bl['WACC'])**5
        data_bl['PV of Terminal Value']=data_bl['Terminal Value']/(1+data_bl['WACC'])**6
        data_bl['Sum of PV of FFCF']=data_bl['PV of FFCF 2024']+data_bl['PV of FFCF 2025']+data_bl['PV of FFCF 2026']+data_bl['PV of FFCF 2027']+data_bl['PV of FFCF 2028']+data_bl['PV of Terminal Value']
        data_bl['Equity Value']=data_bl['Sum of PV of FFCF']+data_bl['Cash/ Cash Equivalent']-data_bl["Total Debt"]
        data_bl['Price per share(Intrinsic Value via DCF)']=data_bl['Equity Value']/data_bl["Share Outstandings"]

        data_bl["Graham Formula / Intrinsic value using EPS"]=(8.5+(data_bl['EPS rate%'])*100)*data_bl['EPS(TTM)']
        data_bl['Over/Undervalue %']=(data_bl['Price']-data_bl['Graham Formula / Intrinsic value using EPS'])/data_bl['Graham Formula / Intrinsic value using EPS']
        try:
            session1 = requests.Session()
            request = session1.get(url,headers=headers)
            request2 = session1.get(url2,headers=headers)
            cookies = dict(request.cookies)
            cookies2 = dict(request2.cookies)
            response = session1.get(url, headers=headers, cookies=cookies).json()
            response2 = session1.get(url2, headers=headers, cookies=cookies2).json()
            rdata_bl = json.dumps(response)
            rdata_bl2 = json.dumps(response2)
            json_data_bl = json.loads(rdata_bl)
            json_data_bl2 = json.loads(rdata_bl2)
            data_bl1 = json_data_bl[0]
            data_bl2 = json_data_bl2[0]
            data_bl['High']=data_bl1['high_target_price']
            data_bl['Median']=data_bl1['median_target_price']
            data_bl['Low']=data_bl1['low_target_price']
            data_bl['Last']=data_bl1['current_stock_price']
            data_bl['Sym']=data_bl1['symbol']
            data_bl['Anal']=data_bl2["num_of_analysts"]
        except:
            pass
        print(data_bl)
        result_bl.append(data_bl)
    #Any error with the previous stock move to next with remarks
    except Exception as e:
        print(e)
        traceback.print_exc()
        data['reason']=e
        result.append(data)
        continue


currently processing: 0
A  agilent-technologies
This event loop is already running


Traceback (most recent call last):
  File "C:\Users\Tanuj\AppData\Local\Temp\ipykernel_26840\1727798718.py", line 12, in <module>
    out=session.run(get_data_summary,get_cashflow,get_stats,get_analysis,get_financials,get_balanceSheet,get_cash_on_hand)
  File "c:\Users\Tanuj\AppData\Local\Programs\Python\Python39\lib\site-packages\requests_html.py", line 774, in run
    done, _ = self.loop.run_until_complete(asyncio.wait(tasks))
  File "c:\Users\Tanuj\AppData\Local\Programs\Python\Python39\lib\asyncio\base_events.py", line 618, in run_until_complete
    self._check_running()
  File "c:\Users\Tanuj\AppData\Local\Programs\Python\Python39\lib\asyncio\base_events.py", line 578, in _check_running
    raise RuntimeError('This event loop is already running')
RuntimeError: This event loop is already running


NameError: name 'data' is not defined

c:\Users\Tanuj\AppData\Local\Programs\Python\Python39\lib\site-packages\pyppeteer\util.py:29: RuntimeWarning: coroutine 'wait' was never awaited
  gc.collect()
Task exception was never retrieved
future: <Task finished name='Task-11' coro=<get_cash_on_hand() done, defined at C:\Users\Tanuj\AppData\Local\Temp\ipykernel_26840\353850992.py:212> exception=UnboundLocalError("local variable 'res' referenced before assignment")>
Traceback (most recent call last):
  File "C:\Users\Tanuj\AppData\Local\Temp\ipykernel_26840\353850992.py", line 214, in get_cash_on_hand
    s=ticker[f"{symbol}"].lower()
NameError: name 'ticker' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Tanuj\AppData\Local\Temp\ipykernel_26840\353850992.py", line 259, in get_cash_on_hand
    print("Cash/cash equivalent error",e,res.content)
UnboundLocalError: local variable 'res' referenced before assignment
[INFO] Starting Chromium downloa

In [12]:

#Converting EPS/DCF data to dataframe
df=pd.DataFrame(result)
df['Over/Undervalue %'] = df['Over/Undervalue %'].apply(lambda x: '{:.2%}'.format(x))
df['Weight of equity(in %)'] = df['Weight of equity(in %)'].apply(lambda x: '{:.2%}'.format(x))
df['Weigh of debt(in %)'] = df['Weigh of debt(in %)'].apply(lambda x: '{:.2%}'.format(x))
df['Cost of equity (in %)'] = df['Cost of equity (in %)'].apply(lambda x: '{:.2%}'.format(x))
df['Market Return (in %)'] = df['Market Return (in %)'].apply(lambda x: '{:.2%}'.format(x))
df['Risk Free rate( in %)'] = df['Risk Free rate( in %)'].apply(lambda x: '{:.2%}'.format(x))
df['Cost of debt after tax(in %)'] = df['Cost of debt after tax(in %)'].apply(lambda x: '{:.2%}'.format(x))
df['Cost of Debt (in %)'] = df['Cost of Debt (in %)'].apply(lambda x: '{:.2%}'.format(x))
df['Effective tax rate (in %)'] = df['Effective tax rate (in %)'].apply(lambda x: '{:.2%}'.format(x))
df['EPS rate%'] = df['EPS rate%'].apply(lambda x: '{:.2%}'.format(x))
df['Return on Equity (ttm)'] = df['Return on Equity (ttm)'].apply(lambda x: '{:.2%}'.format(x))
df['Eps Diluted 2023%'] = df['Eps Diluted 2023%'].apply(lambda x: '{:.2%}'.format(x))
df['Eps Diluted 2022%'] = df['Eps Diluted 2022%'].apply(lambda x: '{:.2%}'.format(x))
df['Eps Diluted 2021%'] = df['Eps Diluted 2021%'].apply(lambda x: '{:.2%}'.format(x))

#converting result for best line method to dataframe
df_bl=pd.DataFrame(result_bl)
df_bl['Over/Undervalue %'] = df_bl['Over/Undervalue %'].apply(lambda x: '{:.2%}'.format(x))
df_bl['Weight of equity(in %)'] = df_bl['Weight of equity(in %)'].apply(lambda x: '{:.2%}'.format(x))
df_bl['Weigh of debt(in %)'] = df_bl['Weigh of debt(in %)'].apply(lambda x: '{:.2%}'.format(x))
df_bl['Cost of equity (in %)'] = df_bl['Cost of equity (in %)'].apply(lambda x: '{:.2%}'.format(x))
df_bl['Market Return (in %)'] = df_bl['Market Return (in %)'].apply(lambda x: '{:.2%}'.format(x))
df_bl['Risk Free rate( in %)'] = df_bl['Risk Free rate( in %)'].apply(lambda x: '{:.2%}'.format(x))
df_bl['Cost of debt after tax(in %)'] = df_bl['Cost of debt after tax(in %)'].apply(lambda x: '{:.2%}'.format(x))
df_bl['Cost of Debt (in %)'] = df_bl['Cost of Debt (in %)'].apply(lambda x: '{:.2%}'.format(x))
df_bl['Effective tax rate (in %)'] = df_bl['Effective tax rate (in %)'].apply(lambda x: '{:.2%}'.format(x))
df_bl['EPS rate%'] = df_bl['EPS rate%'].apply(lambda x: '{:.2%}'.format(x))
df_bl['Return on Equity (ttm)'] = df_bl['Return on Equity (ttm)'].apply(lambda x: '{:.2%}'.format(x))
df_bl['Eps Diluted 2023%'] = df_bl['Eps Diluted 2023%'].apply(lambda x: '{:.2%}'.format(x))
df_bl['Eps Diluted 2022%'] = df_bl['Eps Diluted 2022%'].apply(lambda x: '{:.2%}'.format(x))
df_bl['Eps Diluted 2021%'] = df_bl['Eps Diluted 2021%'].apply(lambda x: '{:.2%}'.format(x))

#==========================================================================================================#
#                                              Saving Data to Excel file                                   #
#==========================================================================================================#
excel_file="DCF_result_full_new_final.xlsx"

# print(df)
# quit()
#Sheet 1
#-------------------------------------Positive EPS-----------------------------#
filtered_df = df[(df['Eps Diluted 2020'] > 0) & (df['Eps Diluted 2021'] > 0) & (df['Eps Diluted 2022'] > 0)& (df['Eps Diluted 2023'] > 0) & (df['Eps Diluted 2024'] > 0)]
filtered_df=filtered_df[filtered_df['reason'].isna()]
# filtered_df=filtered_df.dropna(subset=['reason'])
filtered_df=filtered_df[['Stock Name', 'Symbol', 'Price','PE Ratio','Return on Equity (ttm)','EPS(TTM)','Eps Diluted 2020','Eps Diluted 2021','Eps Diluted 2022','Eps Diluted 2023','Eps Diluted 2024','Graham Formula / Intrinsic value using EPS','Over/Undervalue %','High','Median','Low','Last','Sym','Anal']]
with pd.ExcelWriter(excel_file) as writer:
    # Write the filtered DataFrame to a new sheet named 'PositiveData'
    filtered_df.to_excel(writer, sheet_name='Sheet 1', index=False)
#------------------------------------------------------------------------------#

#Sheet 2
#--------------------------------------DCF Analysis----------------------------#
selected_columns_df = df[df['reason'].isna()]
# selected_columns_df = df
selected_columns_df = df[df['Price per share(Intrinsic Value via DCF)'].notna()]
selected_columns_df = selected_columns_df[['Stock Name', 'Symbol', 'Price','PE Ratio','Return on Equity (ttm)','FCF 2020','FCF 2021','FCF 2022','FCF 2023','Share Outstandings','Perpetual Growth rate','WACC','FFCF 2024','FFCF 2025','FFCF 2026','FFCF 2027','FFCF 2028','Terminal Value','PV of FFCF 2024','PV of FFCF 2025','PV of FFCF 2026','PV of FFCF 2027','PV of FFCF 2028','PV of Terminal Value','Sum of PV of FFCF','Cash/ Cash Equivalent','Total Debt','Equity Value','Price per share(Intrinsic Value via DCF)','High','Median','Low','Last','Sym','Anal']]
# Create an Excel writer object
with pd.ExcelWriter(excel_file,mode='a') as writer:
    # Write the selected columns to a new sheet named 'SelectedColumns'
    selected_columns_df.to_excel(writer, sheet_name='Sheet 2', index=False)
#-----------------------------------------------------------------------------#

#Sheet 3
#--------------------------------Best Line Method---------------------------#
# df_bl=df_bl.dropna(subset=['reason'])
df_bl=df_bl[(df_bl['reason'].isna())|(df_bl['reason']=='negative EPS')]
selected_columns_df1 = df_bl[['Stock Name', 'Symbol', 'Price','PE Ratio','Return on Equity (ttm)','FCF 2020','FCF 2021','FCF 2022','FCF 2023','Share Outstandings','Perpetual Growth rate','WACC','FFCF 2024','FFCF 2025','FFCF 2026','FFCF 2027','FFCF 2028','Terminal Value','PV of FFCF 2024','PV of FFCF 2025','PV of FFCF 2026','PV of FFCF 2027','PV of FFCF 2028','PV of Terminal Value','Sum of PV of FFCF','Cash/ Cash Equivalent','Total Debt','Equity Value','M','Price per share(Intrinsic Value via DCF)','High','Median','Low','Last','Sym','Anal']]
selected_columns_df1.rename(columns={'Price per share(Intrinsic Value via DCF)': 'Price per share(Intrinsic Value via BLM)'}, inplace=True)
# Create an Excel writer object
with pd.ExcelWriter(excel_file,mode='a') as writer:
    # Write the selected columns to a new sheet named 'SelectedColumns'
    selected_columns_df1.to_excel(writer, sheet_name='Sheet 3', index=False)
#----------------------------------------------------------------------------#

#Sheet 4
#---------------------------Comparison sheet---------------------------------#
sheet_names = ['Sheet 1', 'Sheet 2', 'Sheet 3']
dfs = [pd.read_excel(excel_file, sheet_name=name) for name in sheet_names]
for i in range(len(dfs)):
    dfs[i]=dfs[i].drop_duplicates()
combined_df = pd.concat([dfs[0][['Symbol','Graham Formula / Intrinsic value using EPS']].set_index('Symbol'), dfs[1][['Symbol','Price per share(Intrinsic Value via DCF)']].set_index('Symbol'), dfs[2][['Symbol','Stock Name','Price','PE Ratio','Return on Equity (ttm)','Price per share(Intrinsic Value via BLM)']].set_index('Symbol')], axis=1)
combined_df.reset_index(inplace=True)
combined_df=combined_df[['Symbol','Stock Name','Price','PE Ratio','Return on Equity (ttm)','Graham Formula / Intrinsic value using EPS','Price per share(Intrinsic Value via DCF)','Price per share(Intrinsic Value via BLM)']]
with pd.ExcelWriter(excel_file, engine='openpyxl', mode='a') as writer:
    combined_df.to_excel(writer, sheet_name='Sheet 4', index=False)
#Sheet 5
#-----------------------------RAW DATA------------------------------------#
with pd.ExcelWriter(excel_file, mode='a') as writer:
    columns_except_column_to_move = [col for col in df.columns if col != 'reason']

    # Reorder the columns putting 'column_to_move' last
    new_columns_order = columns_except_column_to_move + ['reason']

    # Create a new DataFrame with the columns in the new order
    df_reordered = df[new_columns_order]
    df_reordered.to_excel(writer, sheet_name='RAW_DATA', index=False)
#------------------------------------------------------------------------#


# Sheet 6
#---------------------------------reasoned Data--------------------------#
reasoned_df = df_reordered[df_reordered['reason'].notna()]
with pd.ExcelWriter(excel_file,mode='a') as writer:
    # Write the selected columns to a new sheet named 'SelectedColumns'
    reasoned_df.to_excel(writer, sheet_name='Sheet 6', index=False)
#------------------------------------------------------------------------#






KeyError: 'Over/Undervalue %'